# BERT Sentence Classifyer


# Inference

In [1]:
from typing import Dict
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch


In [ ]:
def load_dataset(fields: List[str], files: Dict[str, str], mapping: Dict[str, int]) -> pd.DataFrame:
    dataset = {}
    
    def encode_label(label: str):
        return mapping[label]

    def load_data(file: str):
        df = pd.read_json(file, lines=True)

        data = []
        for _, i in df.iterrows():
            text = ""
            for field in fields:
                if isinstance(i[field], list):
                    text += ' '.join(i[field])
                elif isinstance(field, str):
                    text += i[field]
                else:
                    raise NotImplemented

            data.append({
                "text": text,
                "label": encode_label(i["tags"][0])})
        return data


    for split in list(files.keys()):
        dataset[split] = load_data(files[split])

    return dataset

In [5]:
fields=["postText", "targetTitle", "targetParagraphs"]
dataset = load_dataset(fields=fields)

In [6]:
texts = [text["text"]for text in dataset["validation"]]
text = texts[0]

### Multy class

In [ ]:

# Multi model prediction
checkpoint = "path to model"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
mapping = {0: 'passage', 1: 'phrase', 2: 'multi'}

def predict_one(text, model):
    tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**tokenized).logits
    predicted_class_id = logits.argmax().item()
    return mapping[predicted_class_id]

predicted_tag = predict_one(text, model)

### One vs. rest

In [ ]:
# one vs rest prediction
checkpoint = {"passage": "path to model", "phrase": "path to model", "multi": "path to model"}
tokenizer = AutoTokenizer.from_pretrained(checkpoint["passage"])
models = {
    "passage": AutoModelForSequenceClassification.from_pretrained(checkpoint["passage"], num_labels=2),
    "phrase": AutoModelForSequenceClassification.from_pretrained(checkpoint["phrase"], num_labels=2),
    "multi": AutoModelForSequenceClassification.from_pretrained(checkpoint["multi"], num_labels=2)
}

def predict_one_multi(text, model):
    def predict_probability(text, model):
        tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            logits = model(**tokenized).logits
        return logits.argmax()


    probabilities = {}
    for tag_name, model in models.items():
        probability = predict_probability(text, model)
        probabilities[tag_name] = probability

    return max(probabilities, key=probabilities.get)


predicted_tag = predict_one_multi(text, models)